In [1]:
import pandas as pd
import numpy as np

# Video One : Marketing Campaign datasets

In [40]:
df = pd.read_csv("../DataSets/datasets.csv")
print(f"Load data with shape: {df.shape[0]} Rows")
print(f"Load data with shape: {df.shape[1]} Columns")

Load data with shape: 2020 Rows
Load data with shape: 12 Columns


In [41]:
# Header cleaning
print(df.columns.tolist())
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
print("FIX Header")
print(df.columns.tolist())

[' Campaign_ID ', 'Campaign_Name', 'Start_Date', 'End_Date', 'Channel', 'Impressions', 'Clicks ', 'Spend', 'Conversions', 'Active', 'Clicks', 'Campaign_Tag']
FIX Header
['campaign_id', 'campaign_name', 'start_date', 'end_date', 'channel', 'impressions', 'clicks', 'spend', 'conversions', 'active', 'clicks', 'campaign_tag']


In [42]:
dirty_spend_marks = df["spend"].astype(str).str.contains(r"\$.")
print(df.loc[dirty_spend_marks, ['campaign_id','spend']].head(10))
df["spend"] = df["spend"].astype(str).str.replace(r"[^\d.-]", "", regex=True)
df["spend"] = pd.to_numeric(df["spend"])
print("FIX Applied")
print(df.loc[dirty_spend_marks, ['campaign_id','spend']].head(10))

   campaign_id     spend
0    CMP-00001   $102.82
21   CMP-00022   $2428.4
22   CMP-00023  $4726.22
31   CMP-00032  $2759.35
32   CMP-00033  $2393.02
36   CMP-00037  $2136.63
37   CMP-00038  $1118.15
40   CMP-00041   $205.48
53   CMP-00054   $919.99
55   CMP-00056  $3219.35
FIX Applied
   campaign_id    spend
0    CMP-00001   102.82
21   CMP-00022  2428.40
22   CMP-00023  4726.22
31   CMP-00032  2759.35
32   CMP-00033  2393.02
36   CMP-00037  2136.63
37   CMP-00038  1118.15
40   CMP-00041   205.48
53   CMP-00054   919.99
55   CMP-00056  3219.35


In [43]:
# Category Typos
print(df["channel"].unique())
cleanup_map = {
    'Facebok':'Facebook',
    'Insta_gram':'Instagram',
    'Gogle':'Google Ads',
    'Tik_Tok':'TikTok',
    'YoTube':'YouTube',
    'N/A': np.nan
}
df["channel"] = df['channel'].replace(cleanup_map)
print("FIX Applied")
print(df["channel"].unique())

['TikTok' 'Facebook' 'Email' 'Instagram' 'Google Ads' 'E-mail' nan 'Gogle'
 'Tik_Tok' 'Facebok' 'Insta_gram']
FIX Applied
['TikTok' 'Facebook' 'Email' 'Instagram' 'Google Ads' 'E-mail' nan]


In [44]:
# handling boolean values
print(df["active"].unique())
bool_map = {
    "Yes": True,
    "Y": True,
    "1": True,
    1: True,
    "No": False,
    "N": False,
    "0": False,
    0: False
}
df["active"] = df["active"].replace(bool_map)
print("FIX Applied")
print(df["active"].unique())

['Y' '0' 'No' 'True' 'Yes' '1' 'False']
FIX Applied
[True False 'True' 'False']


In [45]:
#Date parsing
print(df["start_date"].dtype)
df["start_date"] = pd.to_datetime(df["start_date"], errors='coerce')
df["end_date"] = pd.to_datetime(df["end_date"], errors='coerce')
print("FIX Applied")
print(df["start_date"].dtype)
print(df["end_date"].dtype)

object
FIX Applied
datetime64[ns]
datetime64[ns]


In [46]:
df = df.loc[:, ~df.columns.duplicated()]

In [47]:
# Logical Integrity {clicks vs impressions}
impossible_clicks = df["clicks"] > df["impressions"]
print(df.loc[impossible_clicks, ['campaign_id','clicks','impressions']].head(3))

Empty DataFrame
Columns: [campaign_id, clicks, impressions]
Index: []


In [50]:
# Logical Integrity {Time travel}
time_travel = df["end_date"] < df["start_date"]
print(df.loc[time_travel, ['campaign_id','start_date','end_date']].head(3))

df.loc[time_travel, 'end_date'] = df.loc[time_travel, 'start_date'] + pd.Timedelta(days=30)
print("FIX Applied")
print(df.loc[time_travel, ['campaign_id','start_date','end_date']].head(3))


   campaign_id start_date   end_date
23   CMP-00024 2023-05-06 2023-05-01
54   CMP-00055 2023-09-01 2023-08-27
71   CMP-00072 2023-02-01 2023-01-27
FIX Applied
   campaign_id start_date   end_date
23   CMP-00024 2023-05-06 2023-06-05
54   CMP-00055 2023-09-01 2023-10-01
71   CMP-00072 2023-02-01 2023-03-03


In [56]:
#Handle Outliers
Q1 = df['spend'].quantile(0.25)
Q2 = df['spend'].quantile(0.75)
IQR = Q2 - Q1
upper_limit = Q2 + (3 * IQR)

outliers = df['spend'] > upper_limit
print(df.loc[outliers, ['campaign_id','spend']].head(3))

print("FIX Applied")
df.loc[outliers, 'spend'] = upper_limit
print(df.loc[outliers, ['campaign_id','spend']].head(3))


     campaign_id      spend
789    CMP-00790  500000.00
1443   CMP-01444    8921.51
1460   CMP-01461  500000.00
FIX Applied
     campaign_id      spend
789    CMP-00790  8603.5375
1443   CMP-01444  8603.5375
1460   CMP-01461  8603.5375


In [60]:
# STARTING PARSING (FEATURE EXTRACTION)
print(df["campaign_name"].head(5))

df["season"] = df["campaign_name"].str.extract(r'Q\d_([^_]+)_')

print("FIX Applied")
print(df[["campaign_name", "season"]].head(5))



0         Q4_Summer_CMP-00001
1         Q1_Launch_CMP-00002
2         Q3_Winter_CMP-00003
3    Q1_BlackFriday_CMP-00004
4         Q2_Winter_CMP-00005
Name: campaign_name, dtype: object
FIX Applied
              campaign_name       season
0       Q4_Summer_CMP-00001       Summer
1       Q1_Launch_CMP-00002       Launch
2       Q3_Winter_CMP-00003       Winter
3  Q1_BlackFriday_CMP-00004  BlackFriday
4       Q2_Winter_CMP-00005       Winter


# Video 2: Airbnb_Open_Data

In [117]:
df2 = pd.read_csv("../DataSets/Airbnb_Open_Data.csv")

C:\Users\MRC\AppData\Local\Temp\ipykernel_13128\4134085429.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("../DataSets/Airbnb_Open_Data.csv")


In [54]:
"""
Deleting redundant columns.
Renaming the column
Droping Duplicates
Cleaning Individual columns
Remove the NaN values from the dataset
Check for some more Transformations
"""


'\nDeleting redundant columns.\nRenaming the column\nDroping Duplicates\nCleaning Individual columns\nRemove the NaN values from the dataset\nCheck for some more Transformations\n'

In [55]:
# Deleting redundant columns
columns_to_keep = ['NAME', 'host id', 'host_identity_verified', 'host name',
       'neighbourhood group', 'neighbourhood', 'lat', 'long', 'country',
       'country code', 'instant_bookable', 'cancellation_policy', 'room type',
       'Construction year', 'price', 'service fee', 'minimum nights',
       'number of reviews', 'last review']
columns_to_drop = ['reviews per month',
       'review rate number', 'calculated host listings count',
       'availability 365', 'house_rules', 'license','id']

In [56]:
len(columns_to_keep)
len(columns_to_drop)

7

# 1st Method filtering data 

In [57]:
df2 = df2[columns_to_keep]

In [58]:
df2.shape

(102599, 19)

# 2nd Method Droping Columns

In [41]:
# df2.drop(columns=columns_to_drop)

# Renaming the columns

In [65]:
df2.columns = df2.columns.str.strip().str.upper().str.replace(" ", "_")

# Droping duplicates

In [69]:
# Droping duplicates
df2.duplicated().value_counts()
df2 = df2.drop_duplicates()
df2.duplicated().sum()

np.int64(0)

# Remove the NaN Value in datasets

In [73]:
df2.isna().sum()

NAME                      250
HOST_ID                     0
HOST_IDENTITY_VERIFIED    289
HOST_NAME                 404
NEIGHBOURHOOD_GROUP        29
NEIGHBOURHOOD              16
LAT                         8
LONG                        8
COUNTRY                   532
COUNTRY_CODE              131
INSTANT_BOOKABLE          105
CANCELLATION_POLICY        76
ROOM_TYPE                   0
CONSTRUCTION_YEAR         214
PRICE                     247
SERVICE_FEE               273
MINIMUM_NIGHTS            400
NUMBER_OF_REVIEWS         183
dtype: int64

In [72]:
df2.drop(columns=['LAST_REVIEW'], inplace=True)

In [74]:
df2.dropna(inplace=True)

In [75]:
df2.isna().sum()

NAME                      0
HOST_ID                   0
HOST_IDENTITY_VERIFIED    0
HOST_NAME                 0
NEIGHBOURHOOD_GROUP       0
NEIGHBOURHOOD             0
LAT                       0
LONG                      0
COUNTRY                   0
COUNTRY_CODE              0
INSTANT_BOOKABLE          0
CANCELLATION_POLICY       0
ROOM_TYPE                 0
CONSTRUCTION_YEAR         0
PRICE                     0
SERVICE_FEE               0
MINIMUM_NIGHTS            0
NUMBER_OF_REVIEWS         0
dtype: int64

# Cleaning Indivitual Columns

In [79]:
df2["HOST_IDENTITY_VERIFIED"] = df2["HOST_IDENTITY_VERIFIED"].str.upper()

In [89]:
df2["INSTANT_BOOKABLE"] = df2["INSTANT_BOOKABLE"].apply(lambda x: 1 if x == True else 0)
df2

,level_0,index,NAME,HOST_ID,HOST_IDENTITY_VERIFIED,HOST_NAME,NEIGHBOURHOOD_GROUP,NEIGHBOURHOOD,LAT,LONG,COUNTRY,COUNTRY_CODE,INSTANT_BOOKABLE,CANCELLATION_POLICY,ROOM_TYPE,CONSTRUCTION_YEAR,PRICE,SERVICE_FEE,MINIMUM_NIGHTS,NUMBER_OF_REVIEWS
0,0,0,Clean & quiet apt home by the park,80014485718,UNCONFIRMED,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,US,0,strict,Private room,2020.0,$966,$193,10.0,9.0
1,1,1,Skylit Midtown Castle,52335172823,VERIFIED,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,US,0,moderate,Entire home/apt,2007.0,$142,$28,30.0,45.0
2,2,4,Entire Apt: Spacious Studio/Loft by central park,92037596077,VERIFIED,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,US,0,moderate,Entire home/apt,2009.0,$204,$41,10.0,9.0
3,3,5,Large Cozy 1 BR Apartment In Midtown East,45498551794,VERIFIED,Michelle,Manhattan,Murray Hill,40.74767,-73.97500,United States,US,1,flexible,Entire home/apt,2013.0,$577,$115,3.0,74.0
4,4,7,BlissArtsSpace!,90821839709,UNCONFIRMED,Emma,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,United States,US,0,moderate,Private room,2009.0,"$1,060",$212,45.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99337,99337,102029,Cozy room in a 2B with backyard access,5155155913,UNCONFIRMED,Cristina,Queens,Astoria,40.76987,-73.92679,United States,US,0,flexible,Private room,2018.0,$822,$164,1.0,9.0
99338,99338,102030,An East Williamsburg Delight,71564706996,VERIFIED,Petra,Brooklyn,Bedford-Stuyvesant,40.68335,-73.91222,United States,US,1,moderate,Private room,2015.0,$455,$91,4.0,36.0
99339,99339,102031,Cozy bedroom in williamsburg,83101851929,VERIFIED,Russell,Brooklyn,Williamsburg,40.70870,-73.96710,United States,US,1,flexible,Private room,2020.0,"$1,078",$216,1.0,58.0
99340,99340,102032,Spacious Room w/ 2 Bed,69037598918,UNCONFIRMED,Christine,Queens,Bayside,40.75047,-73.75349,United States,US,0,moderate,Private room,2007.0,$103,$21,7.0,40.0


In [92]:
df2.drop(columns=["index"], inplace=True)

In [95]:
df2.drop(columns=["level_0"], inplace=True)

In [108]:
df2.tail(10)

,NAME,HOST_ID,HOST_IDENTITY_VERIFIED,HOST_NAME,NEIGHBOURHOOD_GROUP,NEIGHBOURHOOD,LAT,LONG,COUNTRY,COUNTRY_CODE,INSTANT_BOOKABLE,CANCELLATION_POLICY,ROOM_TYPE,CONSTRUCTION_YEAR,PRICE,SERVICE_FEE,MINIMUM_NIGHTS,NUMBER_OF_REVIEWS
99332,Comfy Room,76192930914,UNCONFIRMED,Jeremiah,Brooklyn,Crown Heights,40.67270,-73.95505,United States,US,0,flexible,Private room,2010,92.0,18.0,4.0,0.0
99333,COZY Large Private APARTMENT in EAST VILLAGE,97186426683,VERIFIED,Luna,Manhattan,East Village,40.72588,-73.98598,United States,US,0,strict,Entire home/apt,2003,878.0,176.0,6.0,22.0
99334,Lovely APT in the Heart of Bushwick!,26348285918,VERIFIED,Andy,Brooklyn,Bushwick,40.70120,-73.92876,United States,US,1,moderate,Entire home/apt,2007,210.0,42.0,3.0,0.0
99335,Vintage Rainbow Room,46908228851,VERIFIED,Ebony,Brooklyn,Flatbush,40.65140,-73.96160,United States,US,0,strict,Private room,2016,118.0,24.0,3.0,25.0
99336,"Gorgeous condo 20min from NYC,close to the air...",27226581142,UNCONFIRMED,Arthur,Queens,Rego Park,40.72999,-73.86043,United States,US,0,moderate,Entire home/apt,2008,994.0,199.0,2.0,67.0
99337,Cozy room in a 2B with backyard access,5155155913,UNCONFIRMED,Cristina,Queens,Astoria,40.76987,-73.92679,United States,US,0,flexible,Private room,2018,822.0,164.0,1.0,9.0
99338,An East Williamsburg Delight,71564706996,VERIFIED,Petra,Brooklyn,Bedford-Stuyvesant,40.68335,-73.91222,United States,US,1,moderate,Private room,2015,455.0,91.0,4.0,36.0
99339,Cozy bedroom in williamsburg,83101851929,VERIFIED,Russell,Brooklyn,Williamsburg,40.70870,-73.96710,United States,US,1,flexible,Private room,2020,1078.0,216.0,1.0,58.0
99340,Spacious Room w/ 2 Bed,69037598918,UNCONFIRMED,Christine,Queens,Bayside,40.75047,-73.75349,United States,US,0,moderate,Private room,2007,103.0,21.0,7.0,40.0
99341,"Room in Queens, NY, near LGA.",56457739998,VERIFIED,Sonia,Queens,East Elmhurst,40.76245,-73.87938,United States,US,1,strict,Private room,2022,982.0,196.0,1.0,239.0


In [100]:
df2['PRICE'] = df2["PRICE"].str.replace("$","").str.replace(",","").str.strip().astype(float)

In [103]:
df2['SERVICE_FEE'] = df2["SERVICE_FEE"].str.replace("$","").str.replace(",","").str.strip().astype(float)

In [106]:
df2["CONSTRUCTION_YEAR"] = df2["CONSTRUCTION_YEAR"].astype(int)

In [110]:
df2["HOST_ID"] = df2["HOST_ID"].astype(int)

In [115]:
df2["HOST_IDENTITY_VERIFIED"] = df2["HOST_IDENTITY_VERIFIED"].astype("str")

In [116]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99342 entries, 0 to 99341
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NAME                    99342 non-null  object 
 1   HOST_ID                 99342 non-null  int64  
 2   HOST_IDENTITY_VERIFIED  99342 non-null  object 
 3   HOST_NAME               99342 non-null  object 
 4   NEIGHBOURHOOD_GROUP     99342 non-null  object 
 5   NEIGHBOURHOOD           99342 non-null  object 
 6   LAT                     99342 non-null  float64
 7   LONG                    99342 non-null  float64
 8   COUNTRY                 99342 non-null  object 
 9   COUNTRY_CODE            99342 non-null  object 
 10  INSTANT_BOOKABLE        99342 non-null  int64  
 11  CANCELLATION_POLICY     99342 non-null  object 
 12  ROOM_TYPE               99342 non-null  object 
 13  CONSTRUCTION_YEAR       99342 non-null  int64  
 14  PRICE                   99342 non-null